In [233]:
import qiskit
import matplotlib.pyplot as plt

In [234]:
qiskit.__version__

'2.0.2'

In [235]:
from qiskit.quantum_info import Statevector
from qiskit import QuantumCircuit
from qiskit.visualization import array_to_latex

sv1 = Statevector.from_int(0, 4)
sv2 = Statevector.from_int(1, 4)
sv3 = Statevector.from_int(2, 4)
sv4 = Statevector.from_int(3, 4)

circ1 = QuantumCircuit(2)
circ1.h(0)
circ1.cx(0, 1)

res1 = sv1.evolve(circ1)
res2 = sv2.evolve(circ1)
res3 = sv3.evolve(circ1)
res4 = sv4.evolve(circ1)

display(array_to_latex(res1.data))
display(array_to_latex(res2.data))
display(array_to_latex(res3.data))
display(array_to_latex(res4.data))


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [236]:
from qiskit.quantum_info import Statevector
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import array_to_latex, plot_histogram

sv1 = Statevector.from_int(0, 4)
sv2 = Statevector.from_int(1, 4)
sv3 = Statevector.from_int(2, 4)
sv4 = Statevector.from_int(3, 4)

circ1 = QuantumCircuit(2)
circ1.h(0)
circ1.cx(0, 1)

res1 = sv1.evolve(circ1)
res2 = sv2.evolve(circ1)
res3 = sv3.evolve(circ1)
res4 = sv4.evolve(circ1)
sim_backend = AerSimulator()
for i, state in enumerate([res1, res2, res3, res4], start=1):
    circ2 = QuantumCircuit(2, 2)
    circ2.initialize(state.data, [0, 1])
    circ2.cx(0, 1)
    circ2.h(0)
    circ2.barrier([0, 1])
    circ2.measure([0, 1], [0, 1])

    circ2_opt = transpile(circ2, sim_backend)
    job = sim_backend.run(circ2_opt, shots=1024)
    res = job.result()
    counts = res.get_counts(circ2_opt)

    print(f"state {i}:")
    print(counts)


state 1:
{'00': 1024}
state 2:
{'01': 1024}
state 3:
{'10': 1024}
state 4:
{'11': 1024}


In [237]:
import pickle
with open(r'C:\Users\Abhinav Prasad\Downloads\ibm_brisbane_noise_model.pkl', "rb") as f:
    noise_model, basis_gates = pickle.load(f)

In [238]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

qc = QuantumCircuit(2, 2)

qc.h(0)
qc.cx(0, 1)

qc.z(0)
qc.x(0)


qc.cx(0, 1)
qc.h(0)

qc.measure([0, 1], [0, 1])

backend = AerSimulator(noise_model=noise_model, basis_gates=basis_gates)

qc_compiled = transpile(qc, backend)
job = backend.run(qc_compiled, shots=1024)
result = job.result()
counts = result.get_counts()

print(counts)
plot_histogram(counts)
plt.show()


{'11': 976, '01': 20, '10': 28}


In [239]:
def word_to_binary(word):
    binary = ''.join(format(ord(c), '08b') for c in word)
    return binary 
# Remember for each letter in your word it will add 8 more bits so choose a relatively shorter word ~12 letters or less


def binary_to_word(binary_str):
    # Pad with zeros if length not multiple of 8
    padded = binary_str.ljust((len(binary_str) + 7) // 8 * 8, '0')
    return ''.join(chr(int(padded[i:i+8], 2)) for i in range(0, len(padded), 8))

In [240]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from collections import Counter
import pickle

with open(r'C:\Users\Abhinav Prasad\Downloads\ibm_brisbane_noise_model.pkl', "rb") as f:
    noise_model, basis_gates = pickle.load(f)

noisy_backend = AerSimulator(noise_model=noise_model, basis_gates=basis_gates)

def word_to_binary(word):
    return ''.join(format(ord(c), '08b') for c in word)

def binary_to_word(binary_str):
    padded = binary_str.ljust((len(binary_str) + 7) // 8 * 8, '0')
    return ''.join(chr(int(padded[i:i+8], 2)) for i in range(0, len(padded), 8))

def majority_vote(L):
    return Counter(L).most_common(1)[0][0]

message = "abc"
binary_message = word_to_binary(message)
print(f"Original message: {message}")
print(f"Binary message: {binary_message}")

recieved_message_trials = []
num_trials = 10

for trial in range(num_trials):
    recieved_message_bits = ''

    for i in range(0, len(binary_message), 2):
        a, b = int(binary_message[i]), int(binary_message[i+1])
        bit_pair_index = 2 * a + b

        qc = QuantumCircuit(2, 2)
        qc.h(0)
        qc.cx(0, 1)

        if bit_pair_index == 1:
            qc.z(0)
        elif bit_pair_index == 2:
            qc.x(0)
        elif bit_pair_index == 3:
            qc.x(0)
            qc.z(0)

        qc.cx(0, 1)
        qc.h(0)
        qc.barrier()
        qc.measure([0, 1], [0, 1])

        qc_compiled = transpile(qc, noisy_backend)
        job = noisy_backend.run(qc_compiled, shots=1)
        result = job.result()
        counts = result.get_counts(qc_compiled)
        measured_bits = list(counts.keys())[0]

        recieved_message_bits += measured_bits

    recieved_word = binary_to_word(recieved_message_bits)
    recieved_message_trials.append(recieved_word)
    print(f"Trial {trial+1}: {recieved_word}  (binary: {recieved_message_bits})")

final_message = ''
for j in range(len(recieved_message_trials[0])):
    candidates = [word[j] for word in recieved_message_trials]
    final_message += majority_vote(candidates)

print("\nFinal decoded message after majority vote:", final_message)


Original message: abc
Binary message: 011000010110001001100011
Trial 1: abc  (binary: 011000010110001001100011)
Trial 2: abc  (binary: 011000010110001001100011)
Trial 3: pbc  (binary: 011100000110001001100011)
Trial 4: `bc  (binary: 011000000110001001100011)
Trial 5: abc  (binary: 011000010110001001100011)
Trial 6: abs  (binary: 011000010110001001110011)
Trial 7: abc  (binary: 011000010110001001100011)
Trial 8: abc  (binary: 011000010110001001100011)
Trial 9: abc  (binary: 011000010110001001100011)
Trial 10: !bc  (binary: 001000010110001001100011)

Final decoded message after majority vote: abc
